In [1]:
import json
import random
json_file_path = './datasets/ICTES/ICTES.json'
def split_json_dataset(json_file_path, train_ratio=0.8, dev_ratio=0.1, test_ratio=0.1, random_seed=None):
    with open(json_file_path, 'r',encoding="utf-8") as f:
        data = json.load(f)

    if random_seed is not None:
        random.seed(random_seed)
        random.shuffle(data)

    total_samples = len(data)
    train_samples = int(total_samples * train_ratio)
    dev_samples = int(total_samples * dev_ratio)
    test_samples = total_samples - train_samples - dev_samples

    train_data = data[:train_samples]
    dev_data = data[train_samples:train_samples + dev_samples]
    test_data = data[train_samples + dev_samples:]

    return train_data, dev_data, test_data
train_data, dev_data, test_data = split_json_dataset(json_file_path)

#Now you have your data split into train, dev, and test datasets
with open("./datasets/ICTES/ICTES_train.json","w",encoding="utf-8") as f1:
    json.dump(train_data,f1,ensure_ascii=False)
with open("./datasets/ICTES/ICTES_dev.json","w",encoding="utf-8") as f2:
    json.dump(dev_data,f2,ensure_ascii=False)
with open("./datasets/ICTES/ICTES_test.json","w",encoding="utf-8") as f3:
    json.dump(test_data,f3,ensure_ascii=False)

In [ ]:
import json

# 输入文件和输出文件的文件名
input_file_name = './data.json'
output_file_name = './data_l.json'

# 打开输入文件以读取JSON数据
with open(input_file_name, 'r',encoding="utf-8") as input_file:
    # 从输入文件中加载JSON数据
    data = json.load(input_file)

# 打开输出文件以写入JSON数据
with open(output_file_name, 'w',encoding="utf-8") as output_file:
    # 逐行写入JSON数据
    for item in data:
        json.dump(item, output_file,ensure_ascii=False)
        output_file.write('\n')

In [3]:
import json
data_count = 0
cond,status,op,check=0,0,0,0
nn = 0
en =0
with open("./datasets/ICTPE_v2/ICTPE_train.json","r") as f:
    data = json.load(f)
data_count = len(data)
for sentence in data:
    for node in sentence["node_list"]:
        if node["type"] == 'condition':cond=cond+1
        if node["type"] == 'operate':op=op+1
        if node["type"] == 'status':status = status+1
        if node["type"] == 'check': check = check+1
    nn=nn+len(sentence["node_list"])
    en=en+len(sentence["edge_list"])
print(data_count)
print(f"{cond}-{status}-{op}-{check}-{en}-{nn}")

136
167-28-258-2-310-455


In [13]:
import re
import json

# 加载Schema
schema_text = """
{
    "condition_rules": [
        {
            "pattern": "(若|假如|当)(.+?)，",
            "description": "条件语句"
        }
    ],
    "status_rules":[
        {
            "pattern":"，(.+?不再.+?)。",
            "description":"状态语句"
        }
    
    ]

}
"""
schema = json.loads(schema_text)

# 流程文本
process_text = """
若温度高于30度，假如湿度超过70%，当时间在晚上8点到早上6点之间，则系统不再执行操作。
"""

# 在流程文本中查找匹配的条件判断
matches = []

for condition_rule in schema["condition_rules"]:
    condition_pattern = condition_rule["pattern"]
    description = condition_rule["description"]
    condition_matches = re.finditer(condition_pattern, process_text)
    for condition_match in condition_matches:
        matches.append((condition_match.group(2), description))
for condition_rule in schema["status_rules"]:
    condition_pattern = condition_rule["pattern"]
    description = condition_rule["description"]
    condition_matches = re.finditer(condition_pattern, process_text)
    for condition_match in condition_matches:
        matches.append((condition_match.groups(0), description))

# 打印匹配的条件判断
for match in matches:
    condition, description = match
    print(f"匹配条件判断: {condition}")
    print(f"规则描述: {description}")
    print()


匹配条件判断: 温度高于30度
规则描述: 条件语句

匹配条件判断: 湿度超过70%
规则描述: 条件语句

匹配条件判断: 时间在晚上8点到早上6点之间
规则描述: 条件语句

匹配条件判断: ('假如湿度超过70%，当时间在晚上8点到早上6点之间，则系统不再执行操作',)
规则描述: 状态语句



In [2]:
import nltk
from nltk import sent_tokenize, word_tokenize, pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def extract_action_subclauses(text):
    # 分句
    sentences = sent_tokenize(text)

    action_subclauses = []

    for sentence in sentences:
        # 分词并标记词性
        words = word_tokenize(sentence)
        tagged_words = pos_tag(words)

        # 查找包含动词的子句
        for i in range(len(tagged_words)):
            if 'VB' in tagged_words[i][1]:
                action_subclause = ' '.join([word for word, _ in tagged_words[i:]])
                action_subclauses.append(action_subclause)

    return action_subclauses

if __name__ == "__main__":
    text = "小狗在花园里跳跃。她读了一本小说，然后在公园散步。"
    action_subclauses = extract_action_subclauses(text)

    print("包含动作的子句：")
    for subclause in action_subclauses:
        print(subclause)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\26862\AppData\Roaming\nltk_data...


KeyboardInterrupt: 

In [3]:
from transformers import BertTokenizer

# Load the pre-trained Chinese BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

# Define your unknown token
unknown_token = "[UNK]"

# Custom function to tokenize text while replacing English phrases with the unknown token
def tokenize_with_unknown(text):
    tokens = tokenizer.tokenize(text)
    
    # Detect and replace English phrases with the unknown token
    for i, token in enumerate(tokens):
        if all(ord(char) < 128 or char in ["-", "*"] for char in token):
            tokens[i] = unknown_token
    
    return tokens

# Example usage
input_text = "在执行 trigger trap 命令之前，需要设备先执行 snmp-agent trap enable feature-name 命令使能LinkUp和LinkDown告警。"
tokenized_text = tokenize_with_unknown(input_text)
print(tokenized_text)


'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/vocab.txt (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EEED1AD6C8>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/vocab.txt


['在', '执', '行', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '命', '令', '之', '前', '，', '需', '要', '设', '备', '先', '执', '行', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '命', '令', '使', '能', '[UNK]', '和', '[UNK]', '告', '警', '。']


In [4]:
from transformers import BertTokenizer

# Define your unknown token
unknown_token = "[UNK]"

# Create a custom tokenizer class
class CustomBertTokenizer:
    def __init__(self, tokenizer_name_or_path):
        self.chinese_tokenizer = BertTokenizer.from_pretrained(tokenizer_name_or_path)
    
    def tokenize(self, text):
        # Replace English phrases with the unknown token
        for char in ["-", "*"]:
            text = text.replace(char, " ")
        tokens = text.split()
        
        # Tokenize the text using the Chinese BERT tokenizer
        tokenized_text = []
        for token in tokens:
            if all(ord(char) < 128 for char in token):
                tokenized_text.append(unknown_token)
            else:
                sub_tokens = self.chinese_tokenizer.tokenize(token)
                tokenized_text.extend(sub_tokens)
        
        return tokenized_text

# Example usage
custom_tokenizer = CustomBertTokenizer("bert-base-chinese")

input_text = "在执行 trigger trap 命令之前，需要设备先执行 snmp-agent trap enable feature-name 命令使能LinkUp和LinkDown告警。"
tokenized_text = custom_tokenizer.tokenize(input_text)
print(tokenized_text)


'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/vocab.txt (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EEED0AE348>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/vocab.txt


['在', '执', '行', '[UNK]', '[UNK]', '命', '令', '之', '前', '，', '需', '要', '设', '备', '先', '执', '行', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '命', '令', '使', '能', '[UNK]', '和', '[UNK]', '告', '警', '。']


In [7]:
from transformers import BertTokenizer

# Define your unknown token
unknown_token = "[UNK]"

# Create a custom tokenizer class
class CustomBertTokenizer:
    def __init__(self, tokenizer_name_or_path):
        self.chinese_tokenizer = BertTokenizer.from_pretrained(tokenizer_name_or_path)
    
    def tokenize(self, text):
        # Split the text into chunks using Chinese characters as separators
        chunks = []
        current_chunk = ""
        
        for char in text:
            if ord(char) < 128:
                current_chunk += char
            else:
                if current_chunk:
                    chunks.append(current_chunk)
                current_chunk = ""
                chunks.append(char)
        
        if current_chunk:
            chunks.append(current_chunk)
        
        # Tokenize the chunks
        tokenized_text = []
        for chunk in chunks:
            if all(ord(char) < 128 for char in chunk):
                tokenized_text.append(unknown_token)
            else:
                sub_tokens = self.chinese_tokenizer.tokenize(chunk)
                tokenized_text.extend(sub_tokens)
        
        return tokenized_text

# Example usage
custom_tokenizer = CustomBertTokenizer("bert-base-chinese")

input_text = "在执行 trigger trap 命令之前，需要设备先执行 snmp-agent trap enable feature-name 命令使能LinkUp和LinkDown告警。"
input_text2 = "配置 **peer peer-as-check** 命令后"
tokenized_text = custom_tokenizer.tokenize(input_text2)
print(tokenized_text)


'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/vocab.txt (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EEED42D5C8>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/vocab.txt


['配', '置', '[UNK]', '命', '令', '后']


In [8]:
from transformers import BertTokenizer

# Define your unknown token
unknown_token = "[UNK]"

# Create a custom tokenizer class
class CustomBertTokenizer(BertTokenizer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def tokenize(self, text, return_offsets_mapping=False, max_length=None, truncation=False):
        # Replace English phrases with the unknown token
        for char in ["-", "*"]:
            text = text.replace(char, " ")
        tokens = text.split()
        
        # Tokenize the text using the Chinese BERT tokenizer
        tokenized_text = []
        for token in tokens:
            if all(ord(char) < 128 for char in token):
                tokenized_text.append(unknown_token)
            else:
                sub_tokens = super().tokenize(token)
                tokenized_text.extend(sub_tokens)
        
        # If return_offsets_mapping is True, calculate and return the offsets
        if return_offsets_mapping:
            offsets = []
            offset = 0
            for token in tokenized_text:
                offsets.append((offset, offset + len(token)))
                offset += len(token)
            
            return tokenized_text, offsets
        
        return tokenized_text

# Example usage
custom_tokenizer = CustomBertTokenizer.from_pretrained("bert-base-chinese")

input_text = "在执行 trigger trap 命令之前"
tokenized_text, offsets = custom_tokenizer.tokenize(input_text, return_offsets_mapping=True)
print(tokenized_text)
print(offsets)


'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/vocab.txt (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EEED183A48>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/vocab.txt
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'CustomBertTokenizer'.


['在', '执', '行', '[UNK]', '[UNK]', '命', '令', '之', '前']
[(0, 1), (1, 2), (2, 3), (3, 8), (8, 13), (13, 14), (14, 15), (15, 16), (16, 17)]


In [12]:
from transformers import BertTokenizer

# Define your unknown token
unknown_token = "[UNK]"

# Create a custom tokenizer class
class CustomBertTokenizer:
    def __init__(self, tokenizer_name_or_path):
        self.chinese_tokenizer = BertTokenizer.from_pretrained(tokenizer_name_or_path)
    
    def tokenize(self, text, max_length=None, truncation=True):
        # Split the text into chunks using Chinese characters as separators
        chunks = []
        current_chunk = ""
        
        for char in text:
            if ord(char) < 128:
                current_chunk += char
            else:
                if current_chunk:
                    chunks.append(current_chunk)
                current_chunk = ""
                chunks.append(char)
        
        if current_chunk:
            chunks.append(current_chunk)
        
        # Tokenize the chunks
        tokenized_text = []
        for chunk in chunks:
            if all(ord(char) < 128 for char in chunk):
                tokenized_text.append(unknown_token)
            else:
                sub_tokens = self.chinese_tokenizer.tokenize(chunk)
                tokenized_text.extend(sub_tokens)
        
        # Calculate offsets mapping
        offset_mapping = []
        offset = 0
        for token in tokenized_text:
            offset_mapping.append((offset, offset + len(token)))
            offset += len(token)
        
        return tokenized_text, offset_mapping

# Example usage
custom_tokenizer = CustomBertTokenizer("bert-base-chinese")

input_text = "在执行 trigger trap 命令之前"
tokenized_text, offset_mapping = custom_tokenizer.tokenize(input_text, max_length=100, truncation=True)
#offset_mapping2 = custom_tokenizer(input_text, max_length=100, truncation=True)["offset_mapping"]
print(tokenized_text)
print(offset_mapping)


'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/vocab.txt (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EEED185948>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/vocab.txt


['在', '执', '行', '[UNK]', '命', '令', '之', '前']
[(0, 1), (1, 2), (2, 3), (3, 8), (8, 9), (9, 10), (10, 11), (11, 12)]


In [13]:
from transformers import BertTokenizer

# Define your unknown token
unknown_token = "[UNK]"

# Create a custom tokenizer class
class CustomBertTokenizer(BertTokenizer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def encode_plus_with_mapping(self, text, max_length=None, truncation=True):
        # Split the text into chunks using Chinese characters as separators
        chunks = []
        current_chunk = ""
        
        for char in text:
            if ord(char) < 128:
                current_chunk += char
            else:
                if current_chunk:
                    chunks.append(current_chunk)
                current_chunk = ""
                chunks.append(char)
        
        if current_chunk:
            chunks.append(current_chunk)
        
        # Tokenize the chunks
        tokenized_text = []
        offsets = []
        offset = 0
        for chunk in chunks:
            if all(ord(char) < 128 for char in chunk):
                tokenized_text.append(unknown_token)
                offsets.append((offset, offset + len(chunk)))
            else:
                sub_tokens = super().tokenize(chunk)
                tokenized_text.extend(sub_tokens)
                offsets.extend([(offset, offset + len(sub_token)) for sub_token in sub_tokens])
            offset += len(chunk)
        
        # Truncate and pad if necessary
        if max_length is not None and len(tokenized_text) > max_length:
            tokenized_text = tokenized_text[:max_length]
            offsets = offsets[:max_length]
        
        # Create input features
        input_ids = super().convert_tokens_to_ids(tokenized_text)
        attention_mask = [1] * len(input_ids)
        token_type_ids = [0] * len(input_ids)
        
        return {
            "input_ids": input_ids,
            "token_type_ids": token_type_ids,
            "attention_mask": attention_mask,
            "offset_mapping": offsets,
        }

# Example usage
custom_tokenizer = CustomBertTokenizer.from_pretrained("bert-base-chinese")

input_text = "在执行 trigger trap 命令之前"
encoded_text = custom_tokenizer.encode_plus_with_mapping(input_text, max_length=100, truncation=True)
token2char_span_mapping = encoded_text["offset_mapping"]
print(token2char_span_mapping)


'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/vocab.txt (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EEED186888>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/vocab.txt
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'CustomBertTokenizer'.


[(0, 1), (1, 2), (2, 3), (3, 17), (17, 18), (18, 19), (19, 20), (20, 21)]


In [4]:
from utils.util import read_json_by_line,write_json_by_line,append_json_by_line
import json
with open("./datasets/ICTPE_v2/ICTPE_v2.json","r",encoding="utf-8") as f:
    datas =json.load(f)
tfdata=read_json_by_line("./outputs/t2f.json")
for i,d in enumerate(tfdata):
    d["text"]=datas[i]["text"]    
write_json_by_line("./outputs/t2f_revised.json",tfdata)

True

In [8]:
from utils.util import read_json_by_line,write_json_by_line,append_json_by_line
import json
data = read_json_by_line("./outputs/t2f_revised.json")
print(data)
with open("./outputs/t2f_revised_formatted.json","w") as f1:
    json.dump(data,f1,ensure_ascii=False)

[{'id': 0, 'response': '输入:在企业网络和园区网中，\n\tif S2700通过百兆电口接入终端用户:\n\t\t上行通过千兆光口或千兆电口接入汇聚层交换机\n\t\t进而通过千兆捆绑或万兆上联到骨干网络\n\t\t构成万兆骨干、百兆到桌面的企业网全网解决方案\n\t\t\t满足用户高带宽、多业务的需求', 'text': '在企业网络和园区网中，S2700通过百兆电口接入终端用户，上行通过千兆光口或千兆电口接入汇聚层交换机，进而通过千兆捆绑或万兆上联到骨干网络，构成万兆骨干、百兆到桌面的企业网全网解决方案，满足用户高带宽、多业务的需求。'}, {'id': 1, 'response': 'if 需要根据接收报文的接口、源IP地址信息、目的IP地址信息、IP承载的协议类型、TCP的源端口号和目的端口号、ICMP协议类型、ICMP code、源MAC地址、目的MAC地址、ARP报文等内容对报文进行分类:\n\t通过引用定义的ACL来满足\n\t先定义ACL并配置规则\n\t在流分类下配置该命令对报文按照ACL进行流分类\n\t实现对匹配同一流分类的报文进行相同的处理', 'text': '当需要根据接收报文的接口、源IP地址信息、目的IP地址信息、IP承载的协议类型、TCP的源端口号和目的端口号、ICMP协议类型、ICMP code、源MAC地址、目的MAC地址、ARP报文等内容对报文进行分类时，可以通过引用定义的ACL来满足。即先定义ACL并配置规则，然后在流分类下配置该命令对报文按照ACL进行流分类，实现对匹配同一流分类的报文进行相同的处理。'}, {'id': 2, 'response': '创建无线业务三层接口VLANIF30\n创建无线业务三层接口VLANIF40\n配置通过接口地址池为STA分配地址', 'text': '\\# 创建无线业务三层接口VLANIF30、VLANIF40并配置通过接口地址池为STA分配地址。'}, {'id': 3, 'response': 'if 创建静态LSP前已经全局使能MPLS能力:\n\t创建静态LSP', 'text': '创建静态LSP前必须确保已经全局使能MPLS能力。'}, {'id': 4, 'response': '如果完成802.1X接入模板的配置:\n\t执行查看配置信息的命令